In [1]:
import sqlite3
from dataclasses import dataclass

@dataclass
class Customer():
    """Class for keeping track of customer details."""
    id: int
    name: str
    acv: float


class Repository():
    """Class for connecting to and interfacing with a sqlite database."""
    def __init__(self, path):
        self.path = path
        self.con = sqlite3.connect(self.path)
        self.cur = self.con.cursor()
        self.cur.execute('''CREATE TABLE IF NOT EXISTS Customers
                            (id INTEGER PRIMARY KEY, name TEXT, acv NUMERIC)''')
    
    
    def get(self, customer):
        for row in self.cur.execute("SELECT * FROM Customers WHERE id=:id LIMIT 1", {"id": customer.id}):
            print(type(row))
            return row
    
    
    def add(self, customer):
        self.cur.execute("INSERT INTO Customers (id, name, acv) Values (?, ?, ?)", (customer.id, customer.name, customer.acv))
        self.con.commit()
    
    
    def update(self, customer):
        self.cur.execute('''UPDATE Customers SET name = ?, acv = ? WHERE id = ?''', (customer.name, customer.acv, customer.id))
        self.con.commit()
    

    def search(self, search_term):
        """Using a generator, yield the customer(s) that match a given search term."""
        for row in self.cur.execute('''SELECT * FROM Customers WHERE name = ?''', [search_term]):
            yield row

In [2]:
myCustomer1 = Customer(1, 'Amazon', 10000)
myRepo = Repository(':memory:')

In [3]:
myRepo.add(myCustomer1)

In [4]:
myRepo.get(myCustomer1)

<class 'tuple'>


(1, 'Amazon', 10000)

In [5]:
myCustomer1.acv = 20000
myRepo.update(myCustomer1)
myRepo.get(myCustomer1)

<class 'tuple'>


(1, 'Amazon', 20000)

In [6]:
myCustomer2 = Customer(2, 'Apple', 20000)
myCustomer3 = Customer(3, 'Snapple', 30000)
myRepo.add(myCustomer2)
myRepo.add(myCustomer3)
for customer in myRepo.search('Amazon'):
    print(customer)

(1, 'Amazon', 20000)
